In [ ]:
import tensorflow as tf
tf.compat.v1.enable_eager_execution()

In [ ]:
tf.executing_eagerly()

In [ ]:
import pandas as pd
import cv2
import os.path
from PIL import Image
from skimage.io import imread, imsave, imshow, show, imread_collection, imshow_collection
import os, logging
logging.disable(logging.WARNING)
os.environ["TF_CPP_MIN_LOG_LEVEL"] = '3'  
from os import listdir
import glob
import numpy as np
import tensorflow as tf
from matplotlib import pyplot as plt
#import keras_tuner as kt
from sklearn.metrics import confusion_matrix, accuracy_score
from tensorflow import keras
os.environ["TF_GPU_ALLOCATOR"]="cuda_malloc_async"
os.environ["TF_CPP_VMODULE"]="gpu_process_state=10,gpu_cudamallocasync_allocator=10"
a = tf.zeros([], tf.float32)

## Imports libs

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from torch.utils.tensorboard import SummaryWriter
from IPython.display import clear_output
device = "cuda" if torch.cuda.is_available() else "cpu"
%load_ext tensorboard

In [ ]:
def renameImageFiles(folderpath, prefix,fileExtension):
    folder_path = folderpath
    new_prefix = prefix

    for i, file_path in enumerate(glob.glob(folder_path + '*.'+fileExtension)):
        new_file_name = new_prefix + '_' + str(i+1) + '.'+fileExtension
        os.rename(file_path, os.path.join(folder_path, new_file_name))


In [ ]:
!pwd

In [ ]:
path1 = '/home/maxwellsam/Multimodal-COVID19-Images/dataset/CT_COVID/'
prefix1 = 'ct_covid'
path2 = '/home/maxwellsam/Multimodal-COVID19-Images/dataset/CT_NonCOVID/'
prefix2 = 'ct_noncovid'

# renameImageFiles(path1, prefix1,'png')
# renameImageFiles(path2, prefix2,'png')
# renameImageFiles(path2, prefix2,'jpg')

In [ ]:
path3 = '/home/maxwellsam/Multimodal-COVID19-Images/dataset/Shap/CT_COVID/'
prefix3 = 'ct_covid'
path4 = '/home/maxwellsam/Multimodal-COVID19-Images/dataset/Shap/CT_NonCOVID/'
prefix4 = 'ct_noncovid'

In [ ]:
def processImages(imgDirPath, binary_label):
    img_names = list()
    try:
        with os.scandir(imgDirPath) as dirs:
            for entry in dirs:
                img_names.append(entry.name)
    except Exception as e:
        print(f"Error while scanning directory: {e}")
        return None

    all_features = []
    for img in img_names:
        try:
            path = imgDirPath + img
            cv_img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
            if cv_img is None:
                print(f"Error reading image: {path}")
                continue

            cv_img2 = cv2.resize(cv_img, (300, 300), interpolation=cv2.INTER_NEAREST)

            nFeatures = (cv_img2.shape[0] * cv_img2.shape[1])
            features = np.reshape(cv_img2, nFeatures)
            all_features.append(features)
        except Exception as e:
            print(f"Error processing image {img}: {e}")

    if len(all_features) == 0:
        print("No valid images found.")
        return None

    imgs_df = pd.DataFrame(np.array(all_features), index=img_names)
    if binary_label == 0:
        imgs_df['class_label'] = np.zeros((imgs_df.shape[0]), dtype=int)
    else:
        imgs_df['class_label'] = np.ones((imgs_df.shape[0]), dtype=int)

    return imgs_df

In [ ]:
ct_covid_features_df =  processImages(path1,1)#1--> covid-19 positive
ct_noncovid_features_df =  processImages(path2,0)#0 ---> covnid-19 negative

In [ ]:
ct_covid_features_df_shap =  processImages(path3,1)#1--> covid-19 positive
ct_noncovid_features_df_shap =  processImages(path4,0)#0 ---> covnid-19 negative

In [ ]:
cvd_imgs = [ct_noncovid_features_df, ct_covid_features_df]
cvd_imgs_dataset = pd.concat(cvd_imgs)
for i in range(100):
    # shuffle the DataFrame rows
    cvd_imgs_dataset = cvd_imgs_dataset.sample(frac = 1)
# cvd_imgs_dataset_colour = cv2.cvtColor(cvd_imgs_dataset, cv2.COLOR_BGR2RGB)
display(cvd_imgs_dataset)

In [ ]:
cvd_imgs_shap = [ct_noncovid_features_df_shap, ct_covid_features_df_shap]
cvd_imgs_dataset_shap = pd.concat(cvd_imgs_shap)
for i in range(100):
    # shuffle the DataFrame rows
    cvd_imgs_dataset_shap = cvd_imgs_dataset_shap.sample(frac = 1)
# cvd_imgs_dataset_colour = cv2.cvtColor(cvd_imgs_dataset, cv2.COLOR_BGR2RGB)
display(cvd_imgs_dataset_shap)

#### Prepare negative covid images for machine learning ready

In [ ]:
input_data_x_ = ct_noncovid_features_df.iloc[:,:-1].to_numpy().reshape((2173,300,300,1))
#input_data_x = cvd_imgs_dataset.iloc[:,:-1].to_numpy()
from sklearn import preprocessing
label_encoder = preprocessing.LabelEncoder()
ct_noncovid_features_df['output_encode'] = label_encoder.fit_transform(ct_noncovid_features_df['class_label'])
ct_noncovid_features_df
ct_noncovid_features_df = pd.get_dummies(ct_noncovid_features_df, columns =['output_encode'])
##Getting the input_labels and input_features for training and testing model
output_label_y_ = np.array(ct_noncovid_features_df[['class_label']])
# print('Input_x Data: \n{0}'.format(input_data_x))
# print('Output_y Data: \n{0}'.format(output_label_y))
input_data_x_negative_only = input_data_x_
output_label_y_negative_only = output_label_y_
print('Input_x Data Shape: \n{0}'.format(input_data_x_negative_only.shape))
print('Output_y Data Shape: \n{0}'.format(output_label_y_negative_only.shape))

#### Prepare positive covid images for machine learning ready

In [ ]:
input_data_x__ = ct_covid_features_df.iloc[:,:-1].to_numpy().reshape((2476,300,300,1))
#input_data_x = cvd_imgs_dataset.iloc[:,:-1].to_numpy()
from sklearn import preprocessing
label_encoder = preprocessing.LabelEncoder()
ct_covid_features_df['output_encode'] = label_encoder.fit_transform(ct_covid_features_df['class_label'])
ct_covid_features_df
ct_covid_features_df = pd.get_dummies(ct_covid_features_df, columns =['output_encode'])
##Getting the input_labels and input_features for training and testing model
output_label_y__ = np.array(ct_covid_features_df[['class_label']])
# print('Input_x Data: \n{0}'.format(input_data_x))
# print('Output_y Data: \n{0}'.format(output_label_y))
input_data_x_positive_only = input_data_x__
output_label_y_positive_only = output_label_y__
print('Input_x Data Shape: \n{0}'.format(input_data_x_positive_only.shape))
print('Output_y Data Shape: \n{0}'.format(output_label_y_positive_only.shape))

#### Prepare both positive and negetive covid images for machine learning ready

In [ ]:
input_data_x = cvd_imgs_dataset.iloc[:,:-1].to_numpy().reshape((4649,300,300,1))
#input_data_x = cvd_imgs_dataset.iloc[:,:-1].to_numpy()
from sklearn import preprocessing
label_encoder = preprocessing.LabelEncoder()
cvd_imgs_dataset['output_encode'] = label_encoder.fit_transform(cvd_imgs_dataset['class_label'])
cvd_imgs_dataset
cvd_imgs_dataset = pd.get_dummies(cvd_imgs_dataset, columns =['output_encode'])
##Getting the input_labels and input_features for training and testing model
output_label_y = np.array(cvd_imgs_dataset[['output_encode_0','output_encode_1']])
# print('Input_x Data: \n{0}'.format(input_data_x))
# print('Output_y Data: \n{0}'.format(output_label_y))
print('Input_x Data Shape: \n{0}'.format(input_data_x.shape))
print('Output_y Data Shape: \n{0}'.format(output_label_y.shape))

In [ ]:
cvd_imgs_dataset

#### Prepare both positive and negetive covid shap images for machine learning ready

In [ ]:
input_data_x_shap = cvd_imgs_dataset_shap.iloc[:,:-1].to_numpy().reshape((3718,300,300,1))
#input_data_x = cvd_imgs_dataset.iloc[:,:-1].to_numpy()
from sklearn import preprocessing
label_encoder = preprocessing.LabelEncoder()
cvd_imgs_dataset_shap['output_encode'] = label_encoder.fit_transform(cvd_imgs_dataset_shap['class_label'])
cvd_imgs_dataset_shap
cvd_imgs_dataset_shap = pd.get_dummies(cvd_imgs_dataset_shap, columns =['output_encode'])
##Getting the input_labels and input_features for training and testing model
output_label_y_shap = np.array(cvd_imgs_dataset_shap[['output_encode_0','output_encode_1']])
# print('Input_x Data: \n{0}'.format(input_data_x))
# print('Output_y Data: \n{0}'.format(output_label_y))
print('Input_x Data Shape: \n{0}'.format(input_data_x_shap.shape))
print('Output_y Data Shape: \n{0}'.format(output_label_y_shap.shape))

In [ ]:
cvd_imgs_dataset_shap

In [ ]:
from sklearn.model_selection import train_test_split

train_features, test_features, train_labels, test_labels = train_test_split(
    input_data_x, output_label_y, test_size=.20, random_state=42)

In [ ]:
from sklearn.model_selection import train_test_split

train_features_shap, test_features_shap, train_labels_shap, test_labels_shap = train_test_split(
    input_data_x_shap, output_label_y_shap, test_size=.20, random_state=42)

In [ ]:
from sklearn.model_selection import train_test_split

train_features_negative_only, test_features_negative_only, train_labels_negative_only, test_labels_negative_only = train_test_split(
    input_data_x_negative_only, output_label_y_negative_only, test_size=.20, random_state=42)

In [ ]:
from sklearn.model_selection import train_test_split

train_features_positive_only, test_features_positive_only, train_labels_positive_only, test_labels__positive_only = train_test_split(
    input_data_x_positive_only, output_label_y_positive_only, test_size=.20, random_state=42)

In [ ]:
train_features_shap.shape

In [ ]:
train_labels

In [ ]:
print(type(train_features_positive_only[0]))

### Loading the dataset

In [ ]:
import torch
from torch.utils.data import Dataset
from PIL import Image
import numpy as np
from torchvision import transforms

class CustomImageDataset(Dataset):
    def __init__(self, features, labels, transform=None):
        """
        Args:
            features (array-like): Input image features (e.g., images as numpy arrays).
            labels (array-like): Corresponding labels for the images.
            transform (callable, optional): Optional transform to be applied on a sample.
        """
        self.features = features
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        image = self.features[idx]
        label = self.labels[idx]

        # Ensure the image has the correct shape
        if image.ndim == 2:  # Grayscale image (height, width)
            image = np.expand_dims(image, axis=-1)  # Convert to (height, width, 1)
        elif image.ndim == 3 and image.shape[2] == 1:  # If the image is (height, width, 1)
            image = np.squeeze(image, axis=-1)  # Convert to (height, width)

        # Convert to PIL image
        if isinstance(image, np.ndarray):
            if image.ndim == 2:  # Grayscale
                image = Image.fromarray(image)  # Convert from numpy array to PIL Image
            elif image.ndim == 3:  # RGB
                image = Image.fromarray(image.astype('uint8'), 'RGB')

        # Apply transformations if any
        if self.transform:
            image = self.transform(image)

        return image, label


In [ ]:
from torchvision import transforms

# Custom normalization for grayscale images (mean and std for single channel)
# normalize_grayscale = transforms.Normalize(mean=[0.485], std=[0.229])

# transform = transforms.Compose([
#     transforms.ToTensor(),
#     normalize_grayscale,  # Apply normalization only for grayscale images
# ])

# Use a conditional normalization based on the number of channels
def dynamic_normalize(tensor):
    """
    Normalize tensors dynamically based on the number of channels.
    Args:
        tensor: A PyTorch tensor representing the image.
    Returns:
        Normalized tensor.
    """
    if tensor.shape[0] == 3:  # RGB Image
        return transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])(tensor)
    elif tensor.shape[0] == 1:  # Grayscale Image
        return transforms.Normalize(mean=[0.485], std=[0.229])(tensor)
    else:
        raise ValueError(f"Unsupported number of channels: {tensor.shape[0]}")

# Add this dynamic normalization into your transform chain:
transform = transforms.Compose([
    transforms.ToTensor(),
    dynamic_normalize,  # Apply normalization dynamically based on channels
])

In [ ]:
# train_dataset = CustomImageDataset(features=train_features, labels=train_labels, transform=transform)
# #test_dataset = CustomImageDataset(features=test_features, labels=test_labels, transform=transform)

# # Create DataLoader instances
# loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
# #test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)


### Load datset for NonCOVID

In [ ]:
train_dataset_negative = CustomImageDataset(features=train_features_negative_only, labels=train_labels_negative_only, transform=transform)
#test_dataset = CustomImageDataset(features=test_features, labels=test_labels, transform=transform)

# Create DataLoader instances
loader_negative = DataLoader(train_dataset_negative, batch_size=64, shuffle=True)
#test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)


### Load datset for COVID

In [ ]:
train_dataset_positive = CustomImageDataset(features=train_features_positive_only, labels=train_labels_positive_only, transform=transform)
#test_dataset = CustomImageDataset(features=test_features, labels=test_labels, transform=transform)

# Create DataLoader instances
loader_positive = DataLoader(train_dataset_positive, batch_size=64, shuffle=True)
#test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)


In [ ]:
diffusion_model = DiffusionModel(input_size=300*300).to(device)

In [ ]:
diffusion_model

### Building the multimodal

In [ ]:
class Discriminator(nn.Module):
    def __init__(self, input_size=300):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(1, 64, 4, 2, 1),  
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout2d(0.3),

            nn.Conv2d(64, 128, 4, 2, 1),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout2d(0.3),

            nn.Conv2d(128, 256, 4, 2, 1),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(256, 512, 4, 2, 1),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout2d(0.3)
        )

        dummy_input = torch.zeros(1, 1, input_size, input_size)
        conv_output_size = self.conv(dummy_input).view(1, -1).size(1)

        self.fc = nn.Sequential(
            nn.Flatten(),
            nn.Linear(conv_output_size, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.conv(x)
        return self.fc(x)



# Define the Generator
class Generator(nn.Module):
    def __init__(self, z_dim, output_size=300):
        super().__init__()
        intermediate_size = output_size // 8  # Dividing by 8 to match the upscale steps

        self.fc = nn.Sequential(
            nn.Linear(z_dim, 256 * intermediate_size * intermediate_size),
            nn.ReLU(inplace=True),
        )

        self.conv = nn.Sequential(
            nn.ConvTranspose2d(256, 128, 4, 2, 1),  # Output: (128, intermediate_size*2, intermediate_size*2)
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),

            nn.ConvTranspose2d(128, 64, 4, 2, 1),  # Output: (64, intermediate_size*4, intermediate_size*4)
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),

            nn.ConvTranspose2d(64, 1, 4, 2, 1),  # Output: (1, intermediate_size*8, intermediate_size*8)
            nn.Tanh()
        )

    def forward(self, x):
        x = self.fc(x)
        intermediate_size = int((x.size(1) // 256) ** 0.5)
        x = x.view(-1, 256, intermediate_size, intermediate_size)
        out = self.conv(x)
        
        # If the output size is not exactly 300x300, we pad it
        out = F.pad(out, (0, 4, 0, 4))  # Pad width and height by 4
        
        return out




# Define the Diffusion Model
class DiffusionModel(nn.Module):
    def __init__(self, input_size=300 * 300):
        super(DiffusionModel, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_size, 512),
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.ReLU()
        )
        self.decoder = nn.Sequential(
            nn.Linear(256, 512),
            nn.ReLU(),
            nn.Linear(512, input_size),
            nn.Tanh()
        )

    def forward(self, x, noise_level):
        x = x.view(x.size(0), -1)
        x = self.encoder(x)
        noise = torch.randn_like(x) * noise_level
        x = x + noise
        x = self.decoder(x)
        x = x.view(x.size(0), 1, 300, 300)
        return x


def gradient_penalty(discriminator, real_data, fake_data, device):
    alpha = torch.rand(real_data.size(0), 1, 1, 1, device=device)
    interpolates = alpha * real_data + (1 - alpha) * fake_data
    interpolates.requires_grad_(True)
    d_interpolates = discriminator(interpolates)
    fake = torch.ones(d_interpolates.size(), device=device)
    gradients = torch.autograd.grad(
        outputs=d_interpolates,
        inputs=interpolates,
        grad_outputs=fake,
        create_graph=True,
        retain_graph=True,
        only_inputs=True
    )[0]
    gradients = gradients.view(gradients.size(0), -1)
    gradient_penalty = ((gradients.norm(2, dim=1) - 1) ** 2).mean()
    return gradient_penalty

In [ ]:
disc = Discriminator().to(device)

In [ ]:
disc

In [ ]:
gen = Generator(z_dim).to(device)

In [ ]:
gen

In [ ]:
# Hyperparameters
import torch.nn.functional as F
torch.cuda.empty_cache() 
device = "cuda" if torch.cuda.is_available() else "cpu"
lr = 0.0002
z_dim = 100
batch_size = 4
num_epochs = 10
betas = (0.5, 0.999)

# Initialize models
disc = Discriminator().to(device)
gen = Generator(z_dim).to(device)
diffusion_model = DiffusionModel().to(device)
fixed_noise = torch.randn((batch_size, z_dim)).to(device)

# Optimizers
opt_disc = optim.Adam(disc.parameters(), lr=lr, betas=betas)
opt_gen = optim.Adam(gen.parameters(), lr=lr, betas=betas)
criterion = nn.BCELoss()

# TensorBoard
writer_fake = SummaryWriter('runs/CNN_GAN_MNIST/fake')
writer_real = SummaryWriter('runs/CNN_GAN_MNIST/real')
writer_losses = SummaryWriter('runs/CNN_GAN_MNIST/losses')

# Start TensorBoard
%tensorboard --logdir=runs/CNN_GAN_MNIST/

save_dir = "/home/maxwellsam/Multimodal-COVID19-Images/generatedImages/COVID-19_/"
os.makedirs(save_dir, exist_ok=True)


# Training Loop
lambda_gp = 10  # Gradient penalty weight

for epoch in range(num_epochs):
    for batch_idx, (real, _) in enumerate(loader_positive):
        real = real.to(device)
        batch_size = real.shape[0]
        noise = torch.randn(batch_size, z_dim).to(device)
        fake = gen(torch.randn(batch_size, z_dim).to(device))
        print(f"Fake shape after modification: {fake.shape}")


        # Discriminator Loss with Gradient Penalty
        disc_real = disc(real).view(-1)
        disc_fake = disc(fake.detach()).view(-1)
        gp = gradient_penalty(disc, real, fake, device)
        lossD = -torch.mean(disc_real) + torch.mean(disc_fake) + lambda_gp * gp
        
        disc.zero_grad()
        lossD.backward()
        opt_disc.step()

        # Generator Loss
        output = disc(fake).view(-1)
        lossG = -torch.mean(output)

        gen.zero_grad()
        lossG.backward()
        opt_gen.step()

        if batch_idx % 100 == 0:
            print(
                f"Epoch [{epoch}/{num_epochs}] Batch {batch_idx}/{len(loader_positive)} "
                f"Loss D: {lossD:.4f}, Loss G: {lossG:.4f}"
            )

            with torch.no_grad():
                fake = gen(fixed_noise)
                img_grid_fake = torchvision.utils.make_grid(fake, normalize=True)
                img_grid_real = torchvision.utils.make_grid(real, normalize=True)

                writer_fake.add_image("MNIST Fake Images", img_grid_fake, global_step=step)
                writer_real.add_image("MNIST Real Images", img_grid_real, global_step=step)

                for i, fake_img in enumerate(fake):
                    normalized_img = (fake_img + 1) / 2
                    save_path = os.path.join(
                        save_dir, f"epoch_{epoch}_batch_{batch_idx}_image_{i}.png"
                    )
                    torchvision.utils.save_image(normalized_img, save_path)

                writer_losses.add_scalar("Discriminator Loss", lossD, global_step=step)
                writer_losses.add_scalar("Generator Loss", lossG, global_step=step)

                step += 1


In [ ]:
torch.cuda.empty_cache()